In [ ]:
import http
from http import client
import csv
import re
import datetime
import numpy as np
import pandas as pd
from time import sleep

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from matplotlib.gridspec import GridSpec

import matplotlib
matplotlib.use('AGG')
%matplotlib inline

In [ ]:
SMALL_SIZE = 10
MEDIUM_SIZE = 18
BIGGER_SIZE = 24

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
#plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title


In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)

In [ ]:
def get_hashtags(s):
    return [i  for i in s.split() if i.startswith("#") ]

In [ ]:
from wordcloud import WordCloud,STOPWORDS

stopwords= set(STOPWORDS)

new_stopwords=stopwords.union({'RT\'','Follow\'', 'https','co','amp'})
wc = WordCloud(width=800, 
               height=400,
               max_font_size=100, 
               max_words=40,
               stopwords=new_stopwords,
               background_color='black',
               normalize_plurals= True,
              collocations=False)

# Actual Hunt  -- User Approach

## Load Hunt Data

In [ ]:
cd /scratch2/pwarren/hunt/

In [ ]:
stem='blackhammer_Nov8'

In [ ]:
social_studio_tweets=pd.read_csv(stem+'.csv', sep=',', dtype=str)

In [ ]:
#ss2=pd.read_csv('#raisethedegree_0530.csv', sep=',', dtype=str)
#social_studio_tweets=social_studio_tweets.append(ss2)

In [ ]:
social_studio_tweets['EXTERNAL_AUTHOR_ID'].value_counts(dropna=False).head(4)

In [ ]:
t_counts=social_studio_tweets[['ARTICLE_ID','EXTERNAL_AUTHOR_ID']].groupby('EXTERNAL_AUTHOR_ID').count().reset_index().rename(columns={'ARTICLE_ID':'tweet_count','EXTERNAL_AUTHOR_ID':'id_str'})

In [ ]:
t_counts.sort_values('tweet_count', ascending=False).head(4)

In [ ]:
social_studio_accounts_tomatch=social_studio_tweets[social_studio_tweets['EXTERNAL_AUTHOR_ID'].notnull()]['EXTERNAL_AUTHOR_ID'].drop_duplicates()

In [ ]:
social_studio_accounts_tomatch.shape

In [ ]:
import pickle
import json
import tweepy
import os

if not os.path.exists('/home/pwarren/local_trolls/secret_twitter_credentials.pkl'):
    Twitter={}
    Twitter['Consumer Key'] = '9ILpVsLZsmSe3tPiusnbgytL3'
    Twitter['Consumer Secret'] = 'v9083F28IVdXKvE15G4mltyzL5DQXrLczU7osi0ymAPCod5NyS'
    Twitter['Access Token'] = '22011162-yFC0NpbGKE8kpRy2RfxIjkaqn7KnGt6d2lsu0mwPw'
    Twitter['Access Token Secret'] = '0OSRTGHPiK68bJccbRU6O845fz6qw0Hr0E1gbndxKFWca'
    with open('/home/pwarren/local_trolls/secret_twitter_credentials.pkl','wb') as f:
        pickle.dump(Twitter, f)
else:
    Twitter=pickle.load(open('/home/pwarren/local_trolls/secret_twitter_credentials.pkl','rb'))


## Authorizing an application to access Twitter account data

auth = tweepy.OAuthHandler(Twitter['Consumer Key'], Twitter['Consumer Secret'])
auth.set_access_token(Twitter['Access Token'], Twitter['Access Token Secret'])


twitter_api = tweepy.API(auth)


## User Approach

In [ ]:

test =social_studio_accounts_tomatch[0:100].tolist() 
api_users=twitter_api.lookup_users(test)

for i in range(0, len(social_studio_accounts_tomatch), 100):
    test=social_studio_accounts_tomatch[i:i+100].tolist()
    try: api_users.extend(twitter_api.lookup_users(test))
    except tweepy.TweepError:
        print ('TweepError')
        sleep(30)

json_data = [r._json for r in api_users]
api_users_pd = pd.json_normalize(json_data)

In [ ]:
api_users_pd = pd.json_normalize(json_data)

In [ ]:
#json_data = [r._json for r in api_users]
#api_users_pd = pd.json_normalize(json_data)

In [ ]:
#api_users_pd=pd.read_json('#GeorgeFloyd_0529_full.json')

In [ ]:
api_users_pd2=api_users_pd.merge(t_counts, on='id_str', how='left')

In [ ]:
api_users_pd2.head(2)

In [ ]:
api_users_pd2[api_users_pd2['tweet_count']>100].shape

In [ ]:
api_users_pd2[api_users_pd2['statuses_count']>50].shape

In [ ]:
api_users_pd2.to_csv(stem+'_users.csv')

In [ ]:
api_users_pd2['anyfavorites']=api_users_pd2['status.favorite_count']>0

In [ ]:
api_users_pd2['follower_per_update']=(api_users_pd2['followers_count'])/(api_users_pd2['statuses_count'])

In [ ]:
screen2=((api_users_pd2['verified']==False)
        &(api_users_pd2['statuses_count']>100) 
        &(api_users_pd2['default_profile_image']==False) 
        &(api_users_pd2['friends_count']>1000) 
        &(api_users_pd2['statuses_count']<10000) 
        &(api_users_pd2['follower_per_update']>.75) 
        &((api_users_pd2['status.source']=='<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>')|(api_users_pd2['status.source']=='<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>')|(api_users_pd2['status.source']=='<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>')) 
        &(~api_users_pd2['description'].str.contains('marketing', na=False, case=False))
        &(~api_users_pd2['description'].str.contains('writ', na=False, case=False))
        &(~api_users_pd2['description'].str.contains('business', na=False, case=False))
        &(~api_users_pd2['description'].str.contains('author', na=False, case=False))
        &(~api_users_pd2['description'].str.contains('profession', na=False, case=False))
        &(~api_users_pd2['description'].str.contains('service', na=False, case=False))
        &(~api_users_pd2['description'].str.contains('design', na=False, case=False))
        )

In [ ]:
screen3=((api_users_pd2['verified']==False)
        &(api_users_pd2['statuses_count']>100) 
        &(api_users_pd2['default_profile_image']==False) 
        &(api_users_pd2['friends_count']>400) 
        &(api_users_pd2['statuses_count']<10000) 
        &((api_users_pd2['follower_per_update']>.75)|(api_users_pd2['status.source']=='<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>') ) 
        &((api_users_pd2['status.source']=='<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>')|(api_users_pd2['status.source']=='<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>')|(api_users_pd2['status.source']=='<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>')) 
        )

In [ ]:
screen_td=((api_users_pd2['verified']==False)
        &(api_users_pd2['statuses_count']>100) 
        &(api_users_pd2['default_profile_image']==False) 
        &(api_users_pd2['friends_count']>100) 
        &(api_users_pd2['status.source']=='<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>') 
        )

In [ ]:
api_users_pd2['profile']='http://twitter.com/'+api_users_pd2['screen_name']

In [ ]:
print(api_users_pd2[screen3].drop_duplicates(subset=['screen_name'])['profile'].shape)
api_users_pd2[screen3].drop_duplicates(subset=['screen_name'])['profile']

In [ ]:
timing_names=api_users_pd2[screen3].drop_duplicates(subset=['screen_name'])['screen_name'].reset_index(drop=True)

In [ ]:

timing_names

## Simple Timing Graphs

In [ ]:
# API Pull
timing_tweets=[]
for timing_name in timing_names:
    try:timing_tweets.extend(twitter_api.user_timeline(screen_name=timing_name, count=200))
    except: print('Error: '+timing_name)
json_data2 = [r._json for r in timing_tweets]
timing_tweets_pd = pd.json_normalize(json_data2)

In [ ]:
#json_data2 = [r._json for r in timing_tweets]
#timing_tweets_pd = pd.json_normalize(json_data2)

In [ ]:
timing_tweets_pd.shape

In [ ]:
timing_tweets_pd.head(1)

In [ ]:
## Calculate stats for visualization

timing_tweets_pd['date_time']=pd.to_datetime(timing_tweets_pd['created_at'], format="%a %b %d %H:%M:%S +0000 %Y")
timing_tweets_pd['date']=timing_tweets_pd['date_time'].dt.normalize()
#timing_tweets_pd['month']=timing_tweets_pd['date_time'].dt.month
#timing_tweets_pd['week']=timing_tweets_pd['date_time'].dt.week
#timing_tweets_pd['year']=timing_tweets_pd['date_time'].dt.year
timing_tweets_pd['hour']=timing_tweets_pd['date_time'].dt.hour
timing_tweets_pd['hashtags']=timing_tweets_pd['text'].apply(get_hashtags)
timing_tweets_pd['birthday']=pd.to_datetime(timing_tweets_pd['user.created_at'], format="%a %b %d %H:%M:%S +0000 %Y")

In [ ]:
timing_tweets_pd['odd_emoji_iphone']=timing_tweets_pd['text'].str.contains('👇',na=False)|timing_tweets_pd['text'].str.contains('😄',na=False)|timing_tweets_pd['text'].str.contains('🔴',na=False)|timing_tweets_pd['text'].str.contains('🚨',na=False)|timing_tweets_pd['text'].str.contains('➕',na=False)
timing_tweets_pd['odd_emoji_ipad']=timing_tweets_pd['text'].str.contains('😏',na=False)|timing_tweets_pd['text'].str.contains('👏',na=False)|timing_tweets_pd['text'].str.contains('🔴',na=False)|timing_tweets_pd['text'].str.contains('🚨',na=False)|timing_tweets_pd['text'].str.contains('➕',na=False)|timing_tweets_pd['text'].str.contains('👏',na=False)|timing_tweets_pd['text'].str.contains('🔴',na=False)|timing_tweets_pd['text'].str.contains('🚨',na=False)|timing_tweets_pd['text'].str.contains('😍',na=False)
timing_tweets_pd['odd_emoji_android']=timing_tweets_pd['text'].str.contains('🔴',na=False)|timing_tweets_pd['text'].str.contains('➕',na=False)


In [ ]:
timing_tweets_pd['iphone']=timing_tweets_pd['source'].str.contains('<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',na=False)
timing_tweets_pd['ipad']=(timing_tweets_pd['source'].str.contains('<a href="http://twitter.com/#!/download/ipad" rel="nofollow">Twitter for iPad</a>',na=False))
timing_tweets_pd['android']=(timing_tweets_pd['source'].str.contains('<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',na=False)) 

In [ ]:
timing_tweets_pd['odd_emoji']=((timing_tweets_pd['odd_emoji_iphone']&timing_tweets_pd['iphone'])
            |(timing_tweets_pd['odd_emoji_android']&timing_tweets_pd['android'])
            |(timing_tweets_pd['odd_emoji_ipad']&timing_tweets_pd['ipad'])
            )&(timing_tweets_pd['retweeted_status.id_str'].isnull())

In [ ]:
timing_tweets_list=timing_tweets_pd.groupby('user.screen_name')[['date','birthday','user.followers_count','user.friends_count','odd_emoji']].agg(first_date=('date','min'), last_date=('date','max'),birthday=('birthday','first'),following_count=('user.friends_count','first'),follower_count=('user.followers_count','first'),tweet_count=('date','count'),odd_emoji=('odd_emoji','sum')).reset_index()
timing_tweets_list.sort_values('birthday', inplace=True)
timing_tweets_list['days']=(timing_tweets_list['last_date']-timing_tweets_list['first_date'])/np.timedelta64(1, 'D')+1

In [ ]:
# Create distribution across hour of day and client, present hashtags, and some account state
# 100 Accounts per file
# I would love to make this a pdf with clickable links to account, but I can't figure out how 
#

hours=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24]

for j in range(0,len(timing_tweets_list['user.screen_name']),100):
    account_list=timing_tweets_list['user.screen_name'][j:j+100].reset_index(drop=True)
    days=timing_tweets_list['days'][j:j+100].reset_index(drop=True)
    tweets=timing_tweets_list['tweet_count'][j:j+100].reset_index(drop=True)
    fols=timing_tweets_list['follower_count'][j:j+100].reset_index(drop=True)
    odd=timing_tweets_list['odd_emoji'][j:j+100].reset_index(drop=True)
    
    #description_list=timing_tweets_pd[['user.screen_name','user.description']].drop_duplicates()['user.description'][j:j+100].reset_index(drop=True)
    fig, ax=plt.subplots(len(account_list),2,figsize=(10,(4*len(account_list))))
    
    for i in range(0,len(account_list)):
        for type in (timing_tweets_pd[timing_tweets_pd['user.screen_name']==account_list[i]]['source'].drop_duplicates()):
            if type=='<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>':
                ax[i,0].hist(timing_tweets_pd[(timing_tweets_pd['user.screen_name']==account_list[i])&(timing_tweets_pd['source']==type)]['hour'], hours ,alpha=.7, label=type, color='red')
            elif type=='<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>': 
                ax[i,0].hist(timing_tweets_pd[(timing_tweets_pd['user.screen_name']==account_list[i])&(timing_tweets_pd['source']==type)]['hour'], hours ,alpha=.7, label=type, color='skyblue')
            elif type=='<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>': 
                ax[i,0].hist(timing_tweets_pd[(timing_tweets_pd['user.screen_name']==account_list[i])&(timing_tweets_pd['source']==type)]['hour'], hours ,alpha=.7, label=type, color='orange')
            elif type=='<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>': 
                ax[i,0].hist(timing_tweets_pd[(timing_tweets_pd['user.screen_name']==account_list[i])&(timing_tweets_pd['source']==type)]['hour'], hours ,alpha=.7, label=type, color='lightgreen')
            elif type=='<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>': 
                ax[i,0].hist(timing_tweets_pd[(timing_tweets_pd['user.screen_name']==account_list[i])&(timing_tweets_pd['source']==type)]['hour'], hours ,alpha=.7, label=type, color='yellow')
            else:
                ax[i,0].hist(timing_tweets_pd[(timing_tweets_pd['user.screen_name']==account_list[i])&(timing_tweets_pd['source']==type)]['hour'], hours ,alpha=.7, label=type, color='grey')
            ax[i,0].legend()
            ax[i,0].set_title(account_list[i], fontsize=12)
           # plt.title(r"\href{http://www.google.com}{This title links to google}", color='gray')

        Hashtag_Combined = " ".join(timing_tweets_pd[(timing_tweets_pd['user.screen_name']==account_list[i])&(timing_tweets_pd['hashtags'].astype(str)!='[]')]['hashtags'].astype(str))
        
        if Hashtag_Combined!='':
            try:
                wc.generate(Hashtag_Combined)
                ax[i,1].imshow(wc, interpolation="bilinear")
                ax[i,1].axis("off")
            except ValueError:
                print('No Hash')
        ax[i,1].set_title('D: '+str(days[i])+" T: "+str(tweets[i])+" F: "+str(fols[i])+" Odd: "+str(odd[i]), fontsize=12)
      
        
        
    plt.savefig(stem+'_screen3_'+str(j)+'.png')